In [ ]:
!pip install streamlit transformers accelerate torch torchvision pyngrok

  Using cached streamlit-1.44.1-py3-none-any.whl.metadata (8.9 kB)
  Using cached watchdog-6.0.0-py3-none-manylinux2014_x86_64.whl.metadata (44 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using c

In [ ]:
!mkdir -p image_caption_ui


In [ ]:
%%writefile image_caption_ui/app.py
import streamlit as st
from PIL import Image
import torch
from transformers import Blip2Processor, Blip2ForConditionalGeneration

st.set_page_config(layout="centered")
st.title("🖼️ Image Description Generator (Paragraph Style)")

uploaded_file = st.file_uploader("Upload an Image", type=["jpg", "png", "jpeg"])
if uploaded_file:
    image = Image.open(uploaded_file).convert('RGB')
    st.image(image, caption="Uploaded Image", use_column_width=True)

    with st.spinner("Generating paragraph..."):
        processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
        model = Blip2ForConditionalGeneration.from_pretrained(
            "Salesforce/blip2-opt-2.7b",
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        )
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model.to(device)

        inputs = processor(images=image, return_tensors="pt").to(device)
        generated_ids = model.generate(**inputs, max_new_tokens=150)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    st.markdown("### 📝 Description:")
    st.write(generated_text)


Overwriting image_caption_ui/app.py


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2w1hd36VcVnPoPB3slzx0fbgGLD_7hjQPtnmJX4HJcAu3AJF2")


In [ ]:
import threading, time, os
from pyngrok import ngrok

# Kill any existing tunnels
ngrok.kill()

# Start streamlit app in background
def run():
    os.system("streamlit run image_caption_ui/app.py")

thread = threading.Thread(target=run)
thread.start()

# Wait for Streamlit to initialize
time.sleep(5)

# Create public URL
public_url = ngrok.connect(8503)
print(f"🔗 Open your app here: {public_url}")


🔗 Open your app here: NgrokTunnel: "https://ac57-34-124-218-8.ngrok-free.app" -> "http://localhost:8503"
